<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In this notebook, I explore a new technique for synthesizing controller. The gist of this new technique is that: rather than trying to find the closed-loop mappings $\{R, M, N, L\}$, or rather, their impulse responses, directly, we find a convex combination of known controllers that satisfies the specification. 

The expected advantages of this method is mainly computational. Synthesis1--the previous approach--couldn't solve for responses with impulse mappings longer than 374 even for the modest 1-dof problem I throw at it. This is a clear disadvantage.